Author: Chris J. Dallmann \
Affiliation: University of Wuerzburg \
Last revision: 26-May-2025

In [1]:
from simulation_model import default_params as params
import pandas as pd
import numpy as np
import os
import yaml
import operator # For fast sorting
import sys
sys.path.append(os.getcwd()) # Enable relative paths

INFO       Cache size for target 'cython': 33127 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the 'C:\Users\Chris\.cython\brian_extensions' directory. [brian2]


In [4]:
# Load simulation data 
experiment_name = "BDN2"
freq = 150

neurons_excited = [10093] # BDN2
#neurons_excited = [10093, 10107] # BDN2, Web
#neurons_excited = [32815, 31635, 32742, 31078] # DNg12 
#neurons_excited = [32815, 31635, 32742, 31078, 10107] # DNg12, Web 

path_to_completeness = "../data/simulation/completeness_manc_v1.csv"
path_to_class = "../data/simulation/classifications_manc_v1.csv"
path_to_parquet = "../data/simulation/" + experiment_name + "/" + str(freq) + "Hz.parquet"
path_to_csv = "../data/simulation/" + experiment_name + "/" + str(freq) + "Hz.csv"
t_run = 1 # Duration of simulation in seconds, relevant for spike frequency calculation below

df_simulation = pd.read_parquet(path_to_parquet)

In [5]:
# Remove connectome IDs that are not in the completeness.csv
df_completeness = pd.read_csv(path_to_completeness)
neurons_in_model = df_completeness['Unnamed: 0'].tolist()
neurons_excited = [(i) for i in neurons_excited if i in neurons_in_model]
print(len(neurons_excited))

1


In [6]:
# Load connectome annotations
df_class = pd.read_csv(path_to_class)

In [7]:
# Set ranks for sorting
class_rank = {}
class_rank['descending neuron'] = 0
class_rank['intrinsic neuron'] = 1
class_rank['motor neuron'] = 2
class_rank['ascending neuron'] = 3
class_rank['sensory neuron'] = 4
class_rank['efferent neuron'] = 5
class_rank['sensory ascending'] = 6
class_rank['efferent ascending'] = 7

In [8]:
# Create dictionary with simulation data and metadata  
dict_simulation = {}
for neuron in df_simulation["connectome_id"].unique():
    dict_simulation[neuron] = {}
    dict_simulation[neuron]["connectome_id"] = neuron
    
    # Get simulation data 
    df_simulation_neuron = df_simulation[df_simulation["connectome_id"] == neuron]
    trials = df_simulation_neuron.trial.unique().tolist()
    spike_frequency = []
    for trial in trials:
        spikes = df_simulation_neuron[df_simulation_neuron["trial"] == trial]["t"].values
        spike_frequency.append(len(spikes)/t_run)

    dict_simulation[neuron]["spike_frequency"] = np.mean(spike_frequency)
    dict_simulation[neuron]["first_spike"] = spikes[0]

    # MANC
    dict_simulation[neuron]["class"] = df_class["class"][df_class.bodyId == neuron].item()
    dict_simulation[neuron]["predictedNt"] = df_class.predictedNt[df_class.bodyId == neuron].item()
    dict_simulation[neuron]["predictedNtProb"] = df_class.predictedNtProb[df_class.bodyId == neuron].item()
    dict_simulation[neuron]["instance"] = df_class.instance[df_class.bodyId == neuron].item()
    dict_simulation[neuron]["subclass"] = df_class.subclass[df_class.bodyId == neuron].item()
    dict_simulation[neuron]["class_rank"] = class_rank[df_class["class"][df_class.bodyId == neuron].item()]

    if neuron in neurons_excited:
        dict_simulation[neuron]["excited"] = 1
    else:
        dict_simulation[neuron]["excited"] = 0

In [9]:
# Sort dictionary based on spike rate within super class  

# Create list of tuples with relevant sorting information (ID, super class, spike frequency)
sort_list = [(i, dict_simulation[i]["class_rank"], dict_simulation[i]["spike_frequency"]) for i in dict_simulation]

# Sort IDs 
sorted_list = sorted(sort_list, key=operator.itemgetter(2), reverse=True)
sorted_list = sorted(sorted_list, key=operator.itemgetter(1), reverse=False)
sorted_ids = [i[0] for i in sorted_list]

# Sort dictionary
dict_simulation_sorted = {i: dict_simulation[i] for i in sorted_ids}

In [10]:
# Convert dictionary to data frame
df_excel = pd.DataFrame.from_dict(data=dict_simulation_sorted, orient='index')#.reset_index()
df_excel.connectome_id = df_excel.connectome_id.astype(str)
df_excel

,connectome_id,spike_frequency,first_spike,class,predictedNt,predictedNtProb,instance,subclass,class_rank,excited
10093,10093,149.300000,0.0080,descending neuron,acetylcholine,0.957519,TBD,xl,0,1
12820,12820,20.500000,0.0221,descending neuron,acetylcholine,0.963664,TBD,xl,0,0
12791,12791,4.300000,0.0239,descending neuron,acetylcholine,0.959030,TBD,xl,0,0
10210,10210,1.400000,0.3583,descending neuron,gaba,0.585442,TBD,xl,0,0
20995,20995,81.700000,0.0133,intrinsic neuron,gaba,0.904615,IN12B008_T2_R,BR,1,0
...,...,...,...,...,...,...,...,...,...,...
13185,13185,1.000000,0.4911,ascending neuron,acetylcholine,0.915454,AN05A003_T3_R,BA,3,0
11552,11552,1.000000,0.5106,ascending neuron,acetylcholine,0.923039,AN19B017_T1_R,BA,3,0
10394,10394,1.000000,0.0999,ascending neuron,acetylcholine,0.959373,AN04B002_T2_R,IA,3,0
10351,10351,1.000000,0.4320,ascending neuron,acetylcholine,0.881990,AN19A009_A1_R,BA,3,0


In [11]:
# Save as csv
df_excel.to_csv(path_to_csv, index=False, header=True)
print('Results saved as ' + path_to_csv)

Results saved as ../data/simulation/BDN2/150Hz.csv
